#### planet_api_querying.py --- Last updated 2/27/21
Contact: James Sayre, sayrejay@gmail.com

#### Written to most efficiently create a patchwork of satellite images over our target area.

#### Inputs: 
"data/muncodes/shp/MUNICIPIOS.shp" -- INEGI provided shapefile of municipality shapes, municipality codes + names

"Intermediates/search_regions_img_corr.csv" -- produced by create_suitability_index.py, a dataset listing all of the target boxes (each box is 0.008333333 degrees by 0.008333333 degrees, or <a href="https://en.wikipedia.org/wiki/Decimal_degrees">approx</a> 1km sq.) we are interested in (including constraints on agricultural suitability, excluding urban areas, water, etc.) for all of Mexico. Each box is associated with the municipality it falls most within, by the variable "muncode" (i.e. the municipality code). x and y in the variable names represent the lat and lon coordinates, i and j represent the ordering of the boxes in the x and y dimensions respectively, and bid is the unique 

#### Outputs:
For each municipality queried, in "Remote Sensing/Planet/":

1) Creates a folder with the state code (i.e. if the municipality code is 20519), creates the folder "20"

2) Within that folder, creates a subfolder with the municipality code, and within that, an "Input" folder

3) Creates folders for each growing season and year

4) Within each subfolder, creates a csv called mun_boxes_CYCLE_NAME_YEAR.csv which details, for the relevant boxes in "search_regions_img_corr.csv" (i.e. the ones with the municipality code 20519) the correspondence between each box (the unique code for each box in this file is bid) and the satellite image it is contained in for that crop cycle and year (found within image_id). Note that some boxes may have multiple satellite images they are contained in.

5) Within the same subfolder, creates a shapefile called "search_results_CYCLE_NAME_YEAR.shp", which details each of the relevant satellite images for a given municipality and crop cycle. The outline of each of the satellite images is plotted in the image file "search_results_CYCLE_NAME_YEAR.png".

In [15]:
import os
import json
import requests as re
from requests.auth import HTTPBasicAuth
import time
# import regionmask
import geopandas as gpd
import shapely
import numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.ops import cascaded_union

import datetime
import matplotlib.pyplot as plt
import pandas as pd
from packaging import version
# from functools import partial
import multiprocessing


### Directories
base_dir = "~/"
remote_sen_input_dir = os.path.join(base_dir, "Remote Sensing", "Input/")
remote_sen_output_dir = os.path.join(base_dir, "Remote Sensing", "Output/")
image_dir = os.path.join(base_dir, "Remote Sensing", "Images/")

data_dir = os.path.join(base_dir, "data")
intermediate_dir = os.path.join(base_dir, "Intermediates")

### Inputs
mun_shp =  os.path.join(data_dir, "muncodes","shp","MUNICIPIOS.shp")
search_region_satelite_imgs = os.path.join(intermediate_dir,"search_regions_img_corr.csv")

### Params
eng_or_esp = "eng" ###eng for english, else for spanish 
### Fetch planet key stored in local environment so it's not shared with other users
PLANET_API_KEY = os.environ['PL_API_KEY_James']

In [16]:
### Programs
def return_date_id(image_id):
    parts = image_id.split('_')[0]
    year,month, day = parts[:4], parts[4:6], parts[6:]
    return int(month), int(day), int(year)

def gt(dt_str):
    dt, _, us = dt_str.partition(".")
    dt = datetime.datetime.strptime(dt, "%Y-%m-%dT%H:%M:%S")
    us = int(us.rstrip("Z"), 10)
    return dt + datetime.timedelta(microseconds=us)
        
def return_overlap_prcnt(sat_polygon,mun_shp):
    ### Computes the percentage overlap between a gpd dataframe (mun_shp, with one obs) and shapely polygon (sat_polygon)
    df_poly = mun_shp.loc[mun_shp.index[0],'geometry']
    return sat_polygon.intersection(df_poly).area/df_poly.area


def search_planet_images(gteyr,gtemnth,gteday,lteyr,ltemnth,lteday,
                         cloud_coverage=0.65,
                         subsetpoly=None,subsetdf=None, itemtype="PSScene4Band",epsg=4326):
    aoi = {'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::'+str(epsg)}},
     'type': 'Polygon'}

    if subsetpoly != None:
        aoi['coordinates'] = shapely.geometry.mapping(subsetpoly)['coordinates'] ## poly
    else:
        aoi['coordinates'] = shapely.geometry.mapping(subsetdf.loc[subsetdf.index[0],'geometry'])['coordinates'] ##df

    geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": aoi
    }

    # get images acquired within a date range
    date_range_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": datetime.datetime(year=gteyr, month=gtemnth, day=gteday).isoformat()+'.000Z',
        "lte": datetime.datetime(year=lteyr, month=ltemnth, day=lteday).isoformat()+'.000Z'
      }
    }

    # only get images which have <50% cloud coverage
    cloud_cover_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lte": cloud_coverage
      }
    }

    # combine our geo, date, cloud filters
    combined_filter = {
      "type": "AndFilter",
      "config": [geometry_filter, date_range_filter, cloud_cover_filter]
    }

    item_type = itemtype

    # API request object
    search_request = {
      "item_types": [item_type], 
      "filter": combined_filter
    }

    # fire off the POST request
    try:
        search_result = \
          re.post('https://api.planet.com/data/v1/quick-search',
          auth=HTTPBasicAuth(PLANET_API_KEY, ''),
          json=search_request)
        results = [feature for feature in search_result.json()['features']]
    except:
        try:
            search_result = \
              re.post('https://api.planet.com/data/v1/quick-search',
              auth=HTTPBasicAuth(PLANET_API_KEY, ''),
              json=search_request)
            results = [feature for feature in search_result.json()['features']]
        except:
            try:
                time.sleep(5)
                search_result = \
                  re.post('https://api.planet.com/data/v1/quick-search',
                  auth=HTTPBasicAuth(PLANET_API_KEY, ''),
                  json=search_request)

                results = [feature for feature in search_result.json()['features']]
            except:
                results = []
    return results

def build_surrounding_box_df(dataframe,delta=0.008333333, epsg=4326):
    """For given lat/lon coordinates of a target area, builds a shapely polygon of that box and adds as col
    --Inputs: dataframe -- pandas dataframe containing x/lon and y/lat coordinates of target area boxes
    --delta -- float specifying width/height of box (in lat/lon coords)
    -- Outputs: geopandas dataframe, with geometry supplying the shapely box for each target area"""
    ytop, ybottom = dataframe['y']+(delta/2.0), dataframe['y']-(delta/2.0)
    xleft, xright = dataframe['x']-(delta/2.0), dataframe['x']+(delta/2.0)
    dataframe['geometry'] =  Polygon([(xleft, ytop), (xright, ytop), (xright, ybottom), (xleft, ybottom)])
    dataframe.crs = "EPSG:"+str(epsg)
    return dataframe


def return_relevant_shapes(result_list, subsetpoly, subsetdf, already_downloaded_ids=[], return_epsg=0,
                          override_filter=False,below_cc_no_dup_threshold=0.10,med_dup_threshold=0.2):
    shapes = []
    ccs = []
    ses = []
    prcnt_covered_muns = []
    ids = []
    epsgs = []
    
    for result in result_list:
        p_filename = result['id']
        if len(result['geometry']['coordinates'][0]) == 1:
            shape = Polygon(result['geometry']['coordinates'][0][0])
        else:
            shape = Polygon(result['geometry']['coordinates'][0])
        prcnt_covered_mun = return_overlap_prcnt(shape,mun_shp=subsetdf) ### df
        prcnt_covered_poly = shape.intersection(subsetpoly).area/subsetpoly.area ### poly
        cc = result['properties']['cloud_cover']
        se= result['properties']['sun_elevation']
        epsg = result['properties']['epsg_code']
        qual = result['properties']['quality_category']
        if override_filter:
            shapes.append(shape)
            ccs.append(cc)
            prcnt_covered_muns.append(-prcnt_covered_mun)
            ids.append(p_filename)
            ses.append(-se)
            epsgs.append(epsg)
        else:
            if prcnt_covered_poly == 1:
                if qual == "standard":
                    shapes.append(shape)
                    ccs.append(cc)
                    prcnt_covered_muns.append(-prcnt_covered_mun)
                    ids.append(p_filename)
                    ses.append(-se)
                    epsgs.append(epsg)


    gdf = gpd.GeoDataFrame({'id':ids,'cloud_cov':ccs,'prcnt_cove':prcnt_covered_muns,
                            'geometry':shapes,'sun_elev':ses,'epsg':epsgs})
    gdf['sun_elev_round'] = (gdf['sun_elev']/5.0).apply(np.round)
    
    if override_filter:
        gdf = gdf.sort_values(['cloud_cov','sun_elev_round','prcnt_cove'])
    else:
        if return_epsg != 0:
            gdf = gdf[gdf['epsg'] == return_epsg]
        if len(already_downloaded_ids) >= 1:
            if len(gdf[gdf['id'].isin(already_downloaded_ids)]) >= 1:
                gdf = gdf[gdf['id'].isin(already_downloaded_ids)]
            else:
                if len(gdf[gdf['cloud_cov']<= below_cc_no_dup_threshold]) >= 1:
                    gdf = gdf.sort_values(['cloud_cov','sun_elev_round','prcnt_cove']).reset_index(drop=True)[0:1]
                elif len(gdf[gdf['cloud_cov']<= med_dup_threshold]) >= 1:
                    gdf = gdf.sort_values(['cloud_cov','sun_elev_round','prcnt_cove']).reset_index(drop=True)[0:2]
                else:
                    gdf = gdf.sort_values(['cloud_cov','sun_elev_round','prcnt_cove']).reset_index(drop=True)[0:3]
        else:
            if len(gdf[gdf['cloud_cov']<= below_cc_no_dup_threshold]) >= 1:
                gdf = gdf.sort_values(['cloud_cov','sun_elev_round','prcnt_cove']).reset_index(drop=True)[0:1]
            elif len(gdf[gdf['cloud_cov']<= med_dup_threshold]) >= 1:
                gdf = gdf.sort_values(['cloud_cov','sun_elev_round','prcnt_cove']).reset_index(drop=True)[0:2]
            else:
                gdf = gdf.sort_values(['cloud_cov','sun_elev_round','prcnt_cove']).reset_index(drop=True)[0:3]
    
    gdf['prcnt_cove'] = -1.0*gdf['prcnt_cove']
    gdf['sun_elev'] = -1.0*gdf['sun_elev']
    
    gdf.drop('sun_elev_round',1,inplace=True)
    gdf['muncode'] = subsetdf['muncode'][list(subsetdf.index)[0]]
    gdf['mun_name'] = subsetdf['NOM_MUN'][list(subsetdf.index)[0]]
    return gdf

def check_smaller_cover(dataframe,box_poly,already_downloaded_ids=[]):
    """Checks if, for the satellite images detailed in dataframe,
    there exists a smaller number of images that will still cover all of box_poly"""
    if len(dataframe[dataframe['cloud_cov'] > 0.10]) >= 1:
        above_cloud_threshold_df = dataframe[dataframe['cloud_cov'] > 0.10]
    else:
        above_cloud_threshold_df = pd.DataFrame()
    above_cloud_threshold_df['indep_cvrg'] = 0.0
    
    dataframe = dataframe[dataframe['cloud_cov'] <= 0.10]
    dataframe['indep_cvrg'] = 0.0
    duplicate_is = []
    for i in dataframe['geometry'].index:
        intersects= []
        for j in dataframe['geometry'].index:
            if i != j:
                if j not in duplicate_is:
                    overlap_area = dataframe['geometry'][i].intersection(dataframe['geometry'][j]).area
                    if overlap_area > 0.0:
                        intersects.append(str(j))

        for j_id, j in enumerate(intersects):
            if j_id == 0:
                merged_poly = dataframe['geometry'][int(j)]
            else: 
                merged_poly = merged_poly.union(dataframe['geometry'][int(j)])
        if len(intersects) != 0:
            if merged_poly.contains(dataframe['geometry'][i].intersection(box_poly)):
                if len(already_downloaded_ids) > 0:
                    if not dataframe.loc[i,'id'] in already_downloaded_ids:
                        duplicate_is.append(i) 
                else:
                    duplicate_is.append(i)
            else:
                non_indp_cov = merged_poly.intersection(dataframe['geometry'][i].intersection(box_poly)).area
                full_cov = dataframe['geometry'][i].intersection(box_poly).area
                dataframe.loc[i,'indep_cvrg'] =  (full_cov-non_indp_cov)##/box_poly.area
                
    dataframe.drop(duplicate_is, 0, inplace=True)
    dataframe = dataframe.append(above_cloud_threshold_df,sort=False).reset_index(drop=True)
    return dataframe

def plot_sat_img_coverage(mun_outline_df,target_boxes_df,satellite_df,plot_path,lang=eng_or_esp):
    '''Make a plot of the municipality outline, the target boxes, and the satellite assets
    Inputs:
    mun_outline_df -- geopandas dataframe with only observation the municipality and associated shapefile
    target_boxes_df -- geopandas dataframe with the target boxes and associated shapefile
    satellite_df -- geopandas dataframe with associated information on each satellite asset and shapefile
    plot_path -- string pointing to where to plot the image
    lang -- string, looks for either "eng" or else plots in Spanish'''
    fig, ax = plt.subplots(figsize=(7, 7))
    mun_outline_df.plot(ax=ax,color='white', edgecolor='blue', alpha=0.7)
    gpd.GeoDataFrame(target_boxes_df).plot(ax=ax,color='red', edgecolor='white', alpha=0.7)
    satellite_df[satellite_df['cloud_cov'] <= 0.10].plot(ax=ax,color='green', edgecolor='white',alpha=0.4)
    satellite_df[satellite_df['cloud_cov'] > 0.10].plot(ax=ax,color='blue', edgecolor='white',alpha=0.2)
    
    centroid_mun = mun_shape_df['geometry'][mun_shape_df.index[0]].centroid
    xcenter, ycenter = centroid_mun.x, centroid_mun.y
    
    datesrt = str(start_yr)+"-"+str(start_mes)+"-"+str(start_day)
    dateend = str(end_yr)+"-"+str(end_mes)+"-"+str(end_day)
    if lang == "eng":
        plt_text = 'Available satellite images for '+mun_name+', from '+datesrt+' to '+dateend+', '+str(np.round(ycenter,2))+'N '+str(np.round(xcenter,2))+'W '
    else:
        plt_text ='Imágenes satélites disponibles de '+mun_name+' municipio '+datesrt+' a '+dateend+', '+str(np.round(ycenter,2))+'N '+str(np.round(xcenter,2))+'W'

    plt.title(plt_text, fontsize=10)    
    plt.savefig(plot_path, dpi=600)
    

def query_all_boxes(all_satellite_df,mun_satellite_df,mun_box_df,mun_df,
                    start_yr, start_mes, start_day, end_yr, end_mes, end_day,img_fls_dled,force_epsg,
                    muncode,mun_name,below_cc_no_dup_threshold=0.10,med_dup_threshold=0.2):
    """For each box, checks whether 1) it is already covered by a satellite image from a neighboring
    municipality or 2) a satellite image from the same municipality and if not, it queries the Planet
    API for satellite imagery that could cover that box
    -- Inputs: all_satellite_df -- geopandas dataframe with information on the images from other states
    -mun_satellite_df -- geopandas dataframe with information on the images from just the mun of interest
    -- mun_box_df -- geopandas dataframe with the target boxes of interest
    --- Outputs: mun_satellite_df, with images on the mun of interest"""
    print("Querying box:")
    for id_box in mun_box_df.index:
        print(id_box),
        subset_p = mun_box_df.loc[id_box,'geometry']
        ### If box contained in any state wide images, append these first to the list of municipality satellite imgs
        if len(all_satellite_df[all_satellite_df['geometry'].apply(lambda x: x.contains(subset_p))]) >=1:
            shapes = all_satellite_df[all_satellite_df['geometry'].apply(lambda x: x.contains(subset_p))].reset_index(drop=True)
            shapes['muncode'] = muncode
            shapes['mun_name'] = mun_name
            mun_satellite_df = mun_satellite_df.append(shapes,sort=False)
            mun_satellite_df = mun_satellite_df.drop_duplicates(subset='id')
        
        box_in_sat_img_df = mun_satellite_df[mun_satellite_df['geometry'].apply(lambda x: x.contains(subset_p))]
        ### If box contained in 3 or more images, nothing to do here     
        if len(box_in_sat_img_df) >=3:
             pass
        ### If box contained in 2 images and 1 below high cloud threshold pass, else download another image(s)     
        elif len(box_in_sat_img_df) == 2:
            if len(box_in_sat_img_df[box_in_sat_img_df['cloud_cov'] <= med_dup_threshold]) >= 1:
                pass
            else:
                results = search_planet_images(start_yr,start_mes,start_day,end_yr,end_mes,end_day,subsetpoly=subset_p)
                shapes = return_relevant_shapes(results,subset_p,mun_df,img_fls_dled,force_epsg)
                mun_satellite_df = mun_satellite_df.append(shapes,sort=False)
                mun_satellite_df = mun_satellite_df.drop_duplicates(subset='id')
                
        ### If box contained in 1 images and it is below medium cloud threshold pass, else download another image(s) 
        elif len(box_in_sat_img_df) == 1:
            if len(box_in_sat_img_df[box_in_sat_img_df['cloud_cov'] <= below_cc_no_dup_threshold]) >= 1:
                pass
            else:
                results = search_planet_images(start_yr,start_mes,start_day,end_yr,end_mes,end_day,subsetpoly=subset_p)
                shapes = return_relevant_shapes(results,subset_p,mun_df,img_fls_dled,force_epsg)
                mun_satellite_df = mun_satellite_df.append(shapes,sort=False)
                mun_satellite_df = mun_satellite_df.drop_duplicates(subset='id')
        else:
            results = search_planet_images(start_yr,start_mes,start_day,end_yr,end_mes,end_day,subsetpoly=subset_p)
            shapes = return_relevant_shapes(results,subset_p,mun_df,img_fls_dled,force_epsg)
            mun_satellite_df = mun_satellite_df.append(shapes,sort=False)
            mun_satellite_df = mun_satellite_df.drop_duplicates(subset='id')
    return mun_satellite_df

In [ ]:
### Read in shapefile for Mexico
df = gpd.read_file(mun_shp)
### We need to reproject this shapefile into lat/lon coordinates, or whatever other projection you want
df = df.to_crs(epsg=4326)
df['muncode'] = df['CVE_ENT']+df['CVE_MUN']
df['muncode'] = df['muncode'].astype(int)

df['area'] = df.geometry.area 
df.sort_values('muncode', inplace=True)

In [4]:
search_reg_df = pd.read_csv(search_region_satelite_imgs) ### Grid cells for Mexico

In [5]:
### Crop cycles for Mexico
crop_cycles = [(2016, 7, 1, 2016, 7, 20, 'summer_start_16'),(2016, 10, 10, 2016, 11, 1, 'summer_end_16'),
               (2016, 11, 15, 2016, 12, 5,'winter_start_17'),(2017, 2, 10, 2017, 3, 1, 'winter_end_17'),
               (2017, 3, 11, 2017, 4, 1,'spring_start_17'),(2017, 5, 20, 2017, 6, 10, 'spring_end_17'),
               (2017, 6, 20, 2017, 7, 10,'summer_start_17'),(2017, 10, 10, 2017, 11, 1, 'summer_end_17'),
               (2017, 11, 15, 2017, 12, 5,'winter_start_18'),(2018, 2, 10, 2018, 3, 1, 'winter_end_18'),
               (2018, 3, 11, 2018, 4, 1,'spring_start_18'),(2018, 5, 20, 2018, 6, 10, 'spring_end_18'),
               (2018, 6, 20, 2018, 7, 10,'summer_start_18'),(2018, 10, 10, 2018, 11, 1, 'summer_end_18'),
               (2018, 11, 15, 2018, 12, 5,'winter_start_19'),(2019, 2, 10, 2019, 3, 1, 'winter_end_19'),
               (2019, 3, 11, 2019, 4, 1,'spring_start_19'),(2019, 5, 20, 2019, 6, 10, 'spring_end_19'),
               (2019, 6, 20, 2019, 7, 10,'summer_start_19'),(2019, 10, 10, 2019, 11, 1, 'summer_end_19'),
               (2019, 11, 15, 2019, 12, 5,'winter_start_20'),(2020,  2, 10, 2020, 3, 1, 'winter_end_20'),
               (2020, 3, 11, 2020, 4, 1,'spring_start_20'),(2020, 5, 20, 2020, 6, 10, 'spring_end_20'),
               (2020, 6, 20, 2020, 7, 10,'summer_start_20'),(2020, 10, 10, 2020, 11, 1, 'summer_end_20'),
               (2020, 11, 15, 2020, 12, 5,'winter_start_21'),(2021,  2, 10, 2021, 3, 1, 'winter_end_21')]

crop_cycles.reverse()

In [6]:
def run_full_query(list_of_muncodes,crop_cycles=crop_cycles,plot_cov_png = False,
                   force_epsg = 0,image_dir=image_dir,
                   remote_sens_input_dir=remote_sen_input_dir):
    ### Use force_epsg = 0, if you don't want to force having only results for one epsg code
    # e.g. force_epsg = 32647 
    for start_yr, start_mes, start_day, end_yr, end_mes, end_day, ciclo_name in crop_cycles:    
        query_shapes_df = gpd.GeoDataFrame({'geometry':[]})
        for muncode in list_of_muncodes:
            state_code = str(muncode).replace(str(muncode)[-3:],"")+"/"
            fullmuncode = str(muncode)+"/"
            name_col = 'NOM_MUN'
            if os.path.exists(image_dir+state_code):
                img_fls_dled = []
                for r,d,f in os.walk(image_dir+state_code):
                    for file in f:
                        if '_sr.tif' in file:
                            img_fls_dled.append(file)                
                img_fls_dled = [img_fl.split('_sr.tif')[0] for img_fl in img_fls_dled]
            else:
                img_fls_dled = []
            output_fl_dir = remote_sens_input_dir+state_code+fullmuncode+ciclo_name+"/"
            if not os.path.exists(output_fl_dir):
                os.makedirs(output_fl_dir)

            if not os.path.isfile(output_fl_dir+'search_results_'+ciclo_name+'.shp'):
                print("Now running muncode:", muncode, "and cycle:", ciclo_name)
                query_shapes_municipial_df= gpd.GeoDataFrame({'geometry':[]})

                mun_search_df = search_reg_df[search_reg_df['muncode'] == muncode].reset_index(drop=True)
                mun_search_df = mun_search_df.apply(build_surrounding_box_df,axis=1)  ### build shapely box for each target area
                mun_shape_df = df[df['muncode'] == muncode]        
                mun_name = df.loc[list(df[df['muncode'] == muncode].index)[0],name_col]

                query_shapes_municipial_df = query_all_boxes(query_shapes_df,query_shapes_municipial_df,mun_search_df,mun_shape_df,
                                                            start_yr, start_mes, start_day, end_yr, end_mes, end_day,
                                                             img_fls_dled, force_epsg, muncode, mun_name)

                query_shapes_municipial_df = query_shapes_municipial_df.reset_index(drop=True)
                print("Checking smaller cover at:",datetime.datetime.now())
                try:
                    query_shapes_municipial_df = check_smaller_cover(query_shapes_municipial_df,cascaded_union(list(mun_search_df['geometry'])),img_fls_dled)
                    print("Finishing cover for",muncode,"at:",datetime.datetime.now())

                    ### Associate each box with the images that cover it
                    for id_box in list(mun_search_df.index):
                        subset_p = mun_search_df.loc[id_box,'geometry']
                        shapes = query_shapes_municipial_df[query_shapes_municipial_df['geometry'].apply(lambda x: x.intersection(subset_p).area > 0.0)]
                        if len(shapes) >=1:
                            mun_search_df.loc[id_box,'image_id'] = ', '.join(list(shapes['id']))
                except:
                    print("HAD ISSUE WITH SMALLER COVER", len(mun_search_df))

                query_shapes_df = query_shapes_df.append(query_shapes_municipial_df,sort=False)
                query_shapes_df = query_shapes_df.reset_index(drop=True)
                if len(query_shapes_municipial_df) >= 1:
                    print(query_shapes_municipial_df.columns)
                    query_shapes_municipial_df['ciclo'] = ciclo_name
                    query_shapes_municipial_df.to_file(output_fl_dir+'search_results_'+ciclo_name+'.shp',index=False)
                    query_shapes_municipial_df.drop('geometry',1).to_csv(output_fl_dir+'search_results_'+ciclo_name+'.csv',index=False)
                mun_search_df['ciclo'] = ciclo_name
                mun_search_df.to_csv(output_fl_dir+'mun_boxes_'+ciclo_name+'.csv',index=False)

                if plot_cov_png:
                    plot_sat_img_coverage(mun_shape_df,mun_search_df,query_shapes_municipial_df,output_fl_dir+'search_results_'+ciclo_name+'.png')
            else:
                print("Skipping", muncode, "for cycle", ciclo_name)
                query_shapes_municipial_df = gpd.read_file(output_fl_dir+'search_results_'+ciclo_name+'.shp')
                query_shapes_df = query_shapes_df.append(query_shapes_municipial_df,sort=False)


In [1]:
target_muns = [2001, 2002, 2003, 2004, 2005] ### Baja California

In [ ]:
mgr = multiprocessing.Manager()
pool_size = multiprocessing.cpu_count()
split_ids = np.array_split(target_muns, pool_size)
pool = multiprocessing.Pool(processes=pool_size)
pool.map(run_full_query, split_ids)
pool.close()